<a href="https://colab.research.google.com/github/oaldak01/Taxi-Tips_ML_Project/blob/main/Taxi_Tips_ML_Model_NYC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import xgboost as xgb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Taxi Data/Distilled_2023_Yellow_Taxi_Trip_Data.txt')

data.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
4999995,1,05/01/2023 03:01:24 PM,05/01/2023 03:08:56 PM,1.0,1.10,1.0,N,107,137,2,9.3,2.5,0.5,0.00,0.0,1.0,13.30,2.5,0.0
4999996,2,12/16/2023 06:04:34 PM,12/16/2023 06:12:02 PM,1.0,0.72,1.0,N,68,186,1,8.6,0.0,0.5,1.00,0.0,1.0,13.60,2.5,0.0
4999997,2,08/28/2023 04:14:44 PM,08/28/2023 04:24:36 PM,1.0,1.85,1.0,N,43,142,1,12.1,2.5,0.5,4.65,0.0,1.0,23.25,2.5,0.0
4999998,1,06/29/2023 07:14:52 PM,06/29/2023 07:27:39 PM,1.0,2.60,1.0,N,246,143,1,14.9,5.0,0.5,5.30,0.0,1.0,26.70,2.5,0.0
4999999,1,12/18/2023 05:27:45 PM,12/18/2023 05:45:34 PM,1.0,2.50,1.0,N,170,236,1,15.6,5.0,0.5,4.40,0.0,1.0,26.50,2.5,0.0


In [ ]:
data.isna().sum()
len(data)

5000000

In [ ]:
#clean data
data['airport_fee'] = data['airport_fee'].fillna(0)
data['congestion_surcharge'] = data['congestion_surcharge'].fillna(0)
data['passenger_count'] = data['passenger_count'].fillna(1)


In [ ]:
#remove redundant data
data = data.drop(columns=['store_and_fwd_flag'])
data['RatecodeID'] = data['RatecodeID'].fillna(99)


# **TRAIN / TEST SPLIT**

In [ ]:
# return 100% of data shuffled, containerized seed number shuffle, get rid of old index prior shuffle and replace with new shuffle.
data = data.sample(frac=1, random_state=432).reset_index(drop=True)


# set prediction intention

targets = data['tip_amount']
features = data.drop(columns=[
    'tip_amount',
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime'
    ])


# split target and feautres into train / test using SLICING
# features = x, targets = y
# icloc() index first 4,000,000 rows train / last 1,000,000 test
x_train, x_test = features.iloc[:-1000000], features.iloc[-1000000:]
y_train, y_test = targets.iloc[:-1000000], targets.iloc[-1000000:]

len(x_test), len(y_test), len(x_train), len(y_train)

(1000000, 1000000, 4000000, 4000000)

# **Training Loop**

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(device="cpu")

#training (give exam questions)
model.fit(x_train, y_train)
# return answer sheet
predictions = model.predict(x_test)
predictions


array([ 1.7536038,  1.1485028,  3.2545907, ...,  1.5738337, 16.598265 ,
        3.2120404], dtype=float32)

array([ 3.6070746e-01,  7.1771011e+00,  2.6999452e+00, ...,
        2.2586088e+00,  3.4708381e-03, -6.5512406e-03], dtype=float32)

        shape = (1000000,)

the model has now trained through 1,000,000 trips and predicted what it thinks the driver should be paid for each trip, now its time to compare the models prections with that actual targets (y_test)

In [ ]:
# to evaluate this comparison we use the MSE Evalutation metric
# MEAN SQUARED ERROR

In [ ]:
from sklearn.metrics import mean_squared_error

# y test is real data, predictions is our predictred tips.
mse = mean_squared_error(y_test, predictions)
mse

32.04838709946803

### the score of mse is  $1.08 = typical error is either +1.08 more or -1.08 less

# REDUCE ERROR to get accurate predictions, HANDLE ANOMALIES

In [ ]:
# returns data with numbers included (use "number" instead of int becuase of dytpe32 + dtype64,
# there are various forms of numbers then just int)
# use .T to transpose output into vertical view
data.select_dtypes(include="number").agg(['min', 'max', 'mean']).T

,min,max,mean
VendorID,1.00,6.00,1.747576
passenger_count,0.00,9.00,1.364498
trip_distance,0.51,99.84,3.667476
RatecodeID,1.00,99.00,4.286890
PULocationID,1.00,265.00,164.952203
DOLocationID,1.00,265.00,163.570355
payment_type,0.00,4.00,1.157730
fare_amount,0.00,633.00,20.329613
extra,0.00,67.33,1.603564
mta_tax,0.00,4.00,0.496920


In [ ]:
# the data indicates negatives indicating refunds.
# these anomolies are too extrene!

# 1. if negative DROP
# 2. insane trip distaced

data = data[data['trip_distance'] > 0.5]
data = data[data['trip_distance'] < 100]

# ~ means remove
# select all numbers that are .lt(0) = less than 0
# .any if any values are less then 0 = negative = in axis=1/ columns
data = data[~data.select_dtypes("number").lt(0).any(axis=1)]

In [ ]:
# anomoloes still wrong for total amount:
# 0.00	696.95
# min    max
# min must be $3 as that is initial fair charge
# 700 is too much for a 100 mile trip

data = data[data['total_amount'] > 3.7]
data = data[data['total_amount'] < 1000]

In [ ]:
len(data)

4640318

# happy with data lets train again

In [ ]:
# return 100% of data shuffled, containerized seed number shuffle, get rid of old index prior shuffle and replace with new shuffle.
data = data.sample(frac=1, random_state=432).reset_index(drop=True)


# set prediction intention

targets = data['tip_amount']
features = data.drop(columns=[
    'tip_amount',
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime'
    ])


# split target and feautres into train / test using SLICING
# features = x, targets = y
# icloc() index first 4,000,000 rows train / last 1,000,000 test
x_train, x_test = features.iloc[:-1000000], features.iloc[-1000000:]
y_train, y_test = targets.iloc[:-1000000], targets.iloc[-1000000:]

len(x_test), len(y_test), len(x_train), len(y_train)

(1000000, 1000000, 3640318, 3640318)

In [ ]:
from sklearn.metrics import mean_squared_error

# y test is real data, predictions is our predictred tips.
mse = mean_squared_error(y_test, predictions)
mse

0.8822801873875582

# ***we went from 1.08 error rate down to 0.88!***